1.ModernICNet
2.ModernICNet --res drop the res connection after conv_s and ffn_c ffn_v add a res after the whole block
3.ModernTCN --res drop like 2 replace IsometricConv with common Conv1d
4.ICNet --drop channel_dim
5.ModernICNet-BN+LN --replace bn with ln
6.ModernICnet-ConvFNN+MlpFNN --replace ConvFNN with MLP 